In [16]:
    base = 'gdrive/MyDrive/ppat/chi_train/'

#Runtime Info
gpu_info = !nvidia-smi
gpu_info = '\n'.join(gpu_info)
if gpu_info.find('failed') >= 0:
  print('Not connected to a GPU')
else:
  print(gpu_info)
from psutil import virtual_memory
ram_gb = virtual_memory().total / 1e9
print('Your runtime has {:.1f} gigabytes of available RAM\n'.format(ram_gb))
if ram_gb < 20:
  print('Not using a high-RAM runtime')
else:
  print('You are using a high-RAM runtime!')


Mon Dec  5 16:37:33 2022       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 460.32.03    Driver Version: 460.32.03    CUDA Version: 11.2     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla T4            Off  | 00000000:00:04.0 Off |                    0 |
| N/A   69C    P0    32W /  70W |   2872MiB / 15109MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

In [17]:
#setup
from google.colab import drive
drive.mount('/content/gdrive')
!rm -rf dataset
!unzip {base}dataset.zip>/dev/null
#pip install tensorflow scikit-learn imbalanced-learn pandas opencv-python-headless


Drive already mounted at /content/gdrive; to attempt to forcibly remount, call drive.mount("/content/gdrive", force_remount=True).


In [18]:
import numpy as np
import random as rn
import tensorflow as tf
import os
os.environ['PYTHONHASHSEED'] = '0'
np.random.seed(0)
rn.seed(0)
tf.random.set_seed(0)
from datetime import datetime
from glob import glob
from imblearn.over_sampling import RandomOverSampler
from imblearn.under_sampling import RandomUnderSampler
from imblearn.combine import SMOTEENN
from random import randint
import sklearn
from sklearn.preprocessing import LabelEncoder
from sklearn.metrics import f1_score, confusion_matrix, precision_recall_fscore_support
from sklearn.model_selection import StratifiedKFold, StratifiedShuffleSplit
from sklearn.utils.class_weight import compute_class_weight
from tensorflow.keras import Sequential
from tensorflow.keras.applications import MobileNetV3Small, MobileNetV3Large, ResNet50V2
from tensorflow.keras.callbacks import Callback, ModelCheckpoint, ReduceLROnPlateau, EarlyStopping, CSVLogger
from tensorflow.keras.layers import Dense, add, Dropout, Flatten, Conv2D, MaxPooling2D, AveragePooling2D, GlobalAveragePooling2D, BatchNormalization, Activation, Input, Concatenate, Cropping2D, ActivityRegularization, RandomBrightness, RandomContrast, RandomCrop, RandomFlip, RandomHeight, RandomRotation, RandomTranslation, RandomWidth, RandomZoom
from tensorflow.keras.models import load_model, Model
from tensorflow.keras.optimizers import SGD, Adam
from tensorflow.keras.preprocessing.image import load_img, img_to_array, ImageDataGenerator
from tensorflow.keras.utils import Sequence
from tensorflow.keras.utils import to_categorical
import codecs
import cv2
import pandas as pd
import time
from google.colab import runtime
tf.get_logger().setLevel('ERROR')

In [19]:
# Util functions
def log(message):
	print(message)
	file = codecs.open(base+"log.txt", "a", "cp1252", "replace")
	print(message, file=file)

def load(file):
	im = cv2.imread(file)
	if im is None:
		print("Can't load", file)
		return
	return tf.image.resize(im, size[:2])


In [20]:
class DataGenerator(Sequence):
	def __init__(self,x, y=None, batch_size = 1, mode="train", alpha=0.2):
		self.x = x
		self.y = y
		self.mode = mode
		self.alpha = alpha
		self.batch_size = batch_size
		self.img_gen = ImageDataGenerator(zca_whitening=False, rotation_range=20, shear_range=0.2, width_shift_range=0.2, height_shift_range=0.2, zoom_range=0.2, horizontal_flip=True)
		self.on_epoch_end()

	def __len__(self):
		return int(np.ceil(len(self.x) / float(self.batch_size)))

	def on_epoch_end(self):
		self.indexes = np.arange(len(self.x))
		if self.mode == "train": np.random.shuffle(self.indexes)

	def aug(self, x):
		if self.mode == "train" and randint(1,10)>1: x = self.img_gen.random_transform(x)
		return x

	def __getitem__(self, index):
		indexes = self.indexes[index*self.batch_size:(index+1)*self.batch_size]
		x_batch = [imgs[x] for x in self.x[indexes]]
		x_batch = np.array(x_batch)
		#x_batch = self.x[indexes]
		if self.y is not None: y_batch = self.y[indexes]
		#if self.mode == "train": x_batch, y_batch = self.mixup(x_batch, y_batch)
		if self.y is None: return x_batch
		return x_batch, y_batch

	def mixup(self, x, y):
		n = x.shape[0]
		l = np.random.beta(self.alpha, self.alpha, n)
		x_l = l.reshape(n, 1, 1, 1)
		y_l = l.reshape(n, 1)
		#index_l = np.arange(n)
		#np.random.shuffle(index_l)

		x1 = x
		x2 = x[::-1]
		x = x1 * x_l + x2 * (1 - x_l)
		
		y1 = y
		y2 = y[::-1]
		y = y1 * y_l + y2 * (1 - y_l)
		return x, y

	def mixup2(self, x, y):
		n = x.shape[0]
		l = np.random.beta(self.alpha, self.alpha)
		
		xl = x[::-1]
		x = x * l + xl * (1 - l)
		
		yl = y[::-1]
		y = y * l + yl * (1 - l)
		return x, y


In [21]:
class MetricCallback(Callback):
	def __init__(self, x, y, k=0, patience=0, restore_best=True):
		super().__init__()
		self.x = x
		self.y = y
		self.k=k
		self.best_score = 0.0
		self.best_epoch = 0
		self.best_weights = None
		self.patience = patience
		self.restore_best = restore_best

	def restore(self):
		self.model.set_weights(self.best_weights)

	def on_epoch_end(self, epoch, logs={}):
		pred_gen= DataGenerator(self.x, mode="predict")
		preds = self.model.predict(pred_gen, verbose=0)
		final = f1_score(self.y.argmax(-1), preds.argmax(-1), average='micro')
		if final>self.best_score:
			for f in glob("*.h5"): os.remove(f)
			self.model.save(f"model {self.k+1} {epoch+1} f{final:.3f}.h5")
			!rm {base}*.h5
			!cp *.h5 {base}
			log(f"{epoch+1} Best Score: improved from {self.best_score:.3f} to {final:.3f}")
			self.best_score = final
			self.best_epoch = epoch
			self.best_weights = self.model.get_weights()
		else: log(f"{epoch+1} Score: {final:.3f}, Best epoch: {self.best_epoch+1}, {self.best_score:.3f}")

		if self.patience>0 and epoch-self.best_epoch >= self.patience:
			print("Stopping...")
			self.stopped_epoch = epoch
			self.model.stop_training = True
			if self.restore_best: self.restore()
			return

		if not epoch == self.best_epoch and (epoch-self.best_epoch)%int(self.patience/2) == 0:
			old_lr = float(tf.keras.backend.get_value(self.model.optimizer.lr))
			new_lr = old_lr * 0.1
			tf.keras.backend.set_value(self.model.optimizer.lr, new_lr)
			self.restore()
			log(f"Reducing learning rate from {old_lr} to {new_lr}")


In [22]:
# Model
def describe(model):
	for i, layer in enumerate(model.layers):
		if layer.trainable: print(i, layer.name, layer.trainable, layer.input_shape, layer.output_shape)
		if hasattr(layer, "layers"): describe(layer)

def create_model(input_shape, output_shape):
	data_augmentation = Sequential([
		RandomBrightness(0.2),
		RandomContrast(0.2),
		RandomFlip(),
		RandomRotation(0.2),
		RandomTranslation(0.2, 0.2),
		RandomZoom(0.2, 0.2)
	])
	base_model = MobileNetV3Small(include_top=False, input_shape=input_shape, weights='imagenet', pooling="avg")
	base_model.trainable = False
	inputs = Input(shape=input_shape)
	x = data_augmentation(inputs)
	x = base_model(x, training=False)
	x = Dense(256)(x)
	x = BatchNormalization()(x)
	x = Activation("relu")(x)
	x = Dropout(0.2)(x)
	outputs =Dense(output_shape,activation='softmax')(x)
	model=Model(inputs=inputs, outputs=outputs)
	model.compile(optimizer=Adam(1e-3), loss='CategoricalCrossentropy',metrics=['accuracy'])
	describe(model)
	print(model.summary())
	return model


In [23]:
#Imblearn
def balance(x, y):
	print("Balancing samples...")
	ros = RandomOverSampler()
	x, y = ros.fit_resample(x.reshape(-1,1), y)
	# Can't use synthetic for memory limitation
	#smote_enn = SMOTEENN(random_state=0)
	#x, y = smote_enn.fit_resample(x, y)
	x = x.reshape(-1)
	u, c = np.unique(y, return_counts=True)
	names = [le.classes_[l] for l in u]
	dist = list(zip(names, c))
	dist = sorted(dist, key = lambda x: x[1])
	print("Distribution after over sampling:", dist)
	return x, y
	

In [24]:
size = (224, 224, 3)
batch_size = 32
k=0
t_start = time.time()
now = datetime.now()
date_time = now.strftime("%m/%d/%Y, %H:%M:%S")
log(f"Starting at: {date_time}")
log(f"Size: {size}, batch_size: {batch_size}, k_fold: {k}")
log(f"Tensorflow: {tf.version.VERSION}")
log(f"SKLearn: {sklearn.__version__}")


Starting at: 12/05/2022, 16:37:36
Size: (224, 224, 3), batch_size: 32, k_fold: 0
Tensorflow: 2.9.2
SKLearn: 1.0.2


In [25]:
#Load and cleanup
df = pd. read_csv (base+"label.csv")
print(df['label'].value_counts())
search = df['label'].value_counts().reset_index(name="count").query("count < 11")["index"]
print("Removing", search.tolist())
df = df[~df['label'].isin(search)]
df = df.dropna(subset=['label'])
print(df['label'].value_counts())

files = df['image']
files = ["dataset/"+file for file in files]
labels = df['label']
bad = []
imgs = {}
for i in range(len(files)):
	data = load(files[i])
	if data is None: bad.append(i)
	else: imgs[files[i]] = data
x = np.delete(np.array(files), np.array(bad))
y = np.delete(np.array(labels), np.array(bad))
le = LabelEncoder()
y = le.fit_transform(y)


knob                 679
non-interactive      389
remove               293
needle               173
icon                  64
button                53
radio button          35
multiple knobs        31
dropdown              29
multiple elements     25
meter                 24
slider                20
light                 16
arrow                 15
switch                12
unknown               10
other                  8
multiple buttons       8
music note             7
multiple switches      4
confused               2
graph                  1
fader                  1
wheel                  1
scale                  1
Name: label, dtype: int64
Removing ['unknown', 'other', 'multiple buttons', 'music note', 'multiple switches', 'confused', 'graph', 'fader', 'wheel', 'scale']
knob                 679
non-interactive      389
remove               293
needle               173
icon                  64
button                53
radio button          35
multiple knobs        31
dropdown        

In [26]:
#Divide train and test set
sss = StratifiedShuffleSplit(n_splits=10, test_size=0.2)
splits = list(sss.split(np.zeros(y.shape[0]), y))
train_index, test_index = splits[k]
x_train, y_train = x[train_index], y[train_index]
x_test, y_test = x[test_index], y[test_index]
log(f"x_train: {x_train.shape}, y_train:{y_train.shape}, x_test: {x_test.shape}, y_test: {y_test.shape}")
#x_train, y_train = balance(x_train, y_train)
y_train= to_categorical(y_train)
y_test = to_categorical(y_test)
log(f"x_train: {x_train.shape}, y_train:{y_train.shape}, x_test: {x_test.shape}, y_test: {y_test.shape}")


x_train: (1485,), y_train:(1485,), x_test: (372,), y_test: (372,)
x_train: (1485,), y_train:(1485, 15), x_test: (372,), y_test: (372, 15)


In [27]:
#Train
log(f"Training fold {k}...")
model = create_model(size, y_train.shape[1])
train_gen= DataGenerator(x_train, y_train, batch_size=batch_size)
val_gen= DataGenerator(x_test, y_test, batch_size=batch_size, mode="val")
mc = MetricCallback(x_test, y_test, patience=50)
logger = CSVLogger(base+'train.csv')
callbacks = [mc, logger]
weight = compute_class_weight('balanced', classes=np.unique(y), y=y)
weight = dict(zip(np.unique(y), weight))
history = model.fit(train_gen, validation_data=val_gen, epochs=1000, callbacks =callbacks, verbose=2, class_weight=weight) #, workers=multiprocessing.cpu_count(), use_multiprocessing=True)


Training fold 0...
0 input_4 True [(None, 224, 224, 3)] [(None, 224, 224, 3)]
1 sequential_1 True (None, 224, 224, 3) (None, 224, 224, 3)
0 random_brightness_1 True (None, 224, 224, 3) (None, 224, 224, 3)
1 random_contrast_1 True (None, 224, 224, 3) (None, 224, 224, 3)
2 random_flip_1 True (None, 224, 224, 3) (None, 224, 224, 3)
3 random_rotation_1 True (None, 224, 224, 3) (None, 224, 224, 3)
4 random_translation_1 True (None, 224, 224, 3) (None, 224, 224, 3)
5 random_zoom_1 True (None, 224, 224, 3) (None, 224, 224, 3)
3 dense_2 True (None, 576) (None, 256)
4 batch_normalization_1 True (None, 256) (None, 256)
5 activation_1 True (None, 256) (None, 256)
6 dropout_1 True (None, 256) (None, 256)
7 dense_3 True (None, 256) (None, 15)
Model: "model_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_4 (InputLayer)        [(None, 224, 224, 3)]     0         
                                             

In [28]:
#Predict
mc.restore()
pred_gen= DataGenerator(x_test, mode="predict")
preds = model.predict(pred_gen)
score = precision_recall_fscore_support(y_test.argmax(-1), preds.argmax(-1), average='micro')
print(f"Micro Precision: {score[0]:.2f}, Recall: {score[1]:.2f}, F1: {score[2]:.2f}")
score = precision_recall_fscore_support(y_test.argmax(-1), preds.argmax(-1), average='weighted')
print(f"Weighted Precision: {score[0]:.2f}, Recall: {score[1]:.2f}, F1: {score[2]:.2f}")
score = precision_recall_fscore_support(y_test.argmax(-1), preds.argmax(-1), average='macro')
print(f"Macro Precision: {score[0]:.2f}, Recall: {score[1]:.2f}, F1: {score[2]:.2f}")
matrix = confusion_matrix(y_test.argmax(-1), preds.argmax(-1))
print("Confusion Matrix:")
for i,m in enumerate(matrix):
  row = [(le.classes_[j], k) for j, k in enumerate(m) if k>0]
  print(i, le.classes_[i], row)


372/372 [==============================] - 3s 7ms/step
Precision: 0.72, Recall: 0.72, Micro F1: 0.72
Precision: 0.76, Recall: 0.72, Weighted F1: 0.72
Precision: 0.52, Recall: 0.62, Macro F1: 0.54
Confusion Matrix:
0 arrow [('arrow', 2), ('light', 1)]
1 button [('button', 8), ('light', 2), ('remove', 1)]
2 dropdown [('dropdown', 4), ('icon', 1), ('remove', 1)]
3 icon [('arrow', 2), ('icon', 9), ('radio button', 1), ('remove', 1)]
4 knob [('knob', 135), ('remove', 1)]
5 light [('button', 2), ('light', 1)]
6 meter [('arrow', 1), ('meter', 3), ('remove', 1)]
7 multiple elements [('meter', 1), ('multiple elements', 3), ('radio button', 1)]
8 multiple knobs [('knob', 1), ('multiple knobs', 4), ('remove', 1)]
9 needle [('needle', 35)]
10 non-interactive [('arrow', 3), ('button', 2), ('dropdown', 4), ('icon', 3), ('knob', 1), ('meter', 5), ('multiple elements', 2), ('needle', 1), ('non-interactive', 36), ('radio button', 2), ('remove', 17), ('slider', 1), ('switch', 1)]
11 radio button [('butt

In [ ]:
#End
now = datetime.now()
date_time = now.strftime("%m/%d/%Y, %H:%M:%S")
log(f"Ending at: {date_time}")
t_finish = time.time()
total_time = (t_finish-t_start)/60
log(f"It took {total_time:.2f} minutes")
time.sleep(1)
runtime.unassign()

Ending at: 12/05/2022, 17:28:17
It took 50.67 minutes
